In [1]:
import logging 
logging.getLogger("py4j").setLevel(logging.ERROR)



In [2]:
import numpy as np
import pandas as pd

!pip install requests
!pip install urllib

import requests
import urllib
from urllib.parse import urlparse

import datetime
from datetime import datetime

opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.0.1 
 Master 
 spark://10.125.226.210:7077 
 AppName 
 Databricks Shell

In [3]:
# urllib.request.urlretrieve('https://raw.githubusercontent.com/adichiara/DS504/main/source_data.csv', '/dbfs/saved_df/source_data.csv')

In [4]:
%sh -e rm -r DS504  

In [5]:
%sh git clone https://github.com/adichiara/DS504

Cloning into 'DS504'...
/bin/bash: line 1: fg: no job control

In [6]:
%sh -e ls DS504/ -al 

total 28
drwxr-xr-x 5 root root 4096 Oct 27 17:49 .
drwxr-xr-x 1 root root 4096 Oct 27 17:49 ..
drwxr-xr-x 2 root root 4096 Oct 27 17:49 data
drwxr-xr-x 8 root root 4096 Oct 27 17:49 .git
drwxr-xr-x 2 root root 4096 Oct 27 17:49 notebooks
-rw-r--r-- 1 root root 7 Oct 27 17:49 README.md
-rw-r--r-- 1 root root 685 Oct 27 17:49 source_data.csv

In [7]:
%fs mv 'file:/databricks/driver/DS504/source_data.csv' '/saved_df/source_data.csv'

res21: Boolean = true

In [8]:
source_df = pd.read_csv("/dbfs/saved_df/source_data.csv", header=0)
display(source_df)

source_name,source_id,source_bias
ABC News,abc-news,Lean Left
Associated Press,associated-press,Center
Axios,axios,Center
Bloomberg,bloomberg,Lean Left
Breitbart News,breitbart-news,Right
Buzzfeed,buzzfeed,Left
CBC News,cbc-news,Lean Left
CNN,cnn,Lean Left
MSNBC,msnbc,Left
National Review,national-review,Right


In [9]:
source_id_list = list(source_df['source_id'])


Out[71]: list

In [10]:
source_id_list = ['abc-news ',
                  'associated-press ',
                  'axios ',
                  'bloomberg ',
                  'breitbart-news ',
                  'buzzfeed ',
                  'cbc-news ',
                  'cnn ',
                  'fox-news',
                  'msnbc ',
                  'national-review ',
                  'nbc-news ',
                  'politico ',
                  'reuters ',
                  'the-hill ',
                  'the-huffington-post ',
                  'the-wall-street-journal ',
                  'the-washington-post ',
                  'the-washington-times ',
                  'usa-today ']

source_id_str = str(source_id_list)[1:len(str(source_id_list))-1].replace("'","")
print(source_id_str)


abc-news , associated-press , axios , bloomberg , breitbart-news , buzzfeed , cbc-news , cnn , fox-news, msnbc , national-review , nbc-news , politico , reuters , the-hill , the-huffington-post , the-wall-street-journal , the-washington-post , the-washington-times , usa-today

In [11]:

url = 'http://newsapi.org/v2/everything?'

def getArticleData(q, source_id, from_date, to_date, limit):
  
  search_params = {
    'language':'en',
    'sortBy':'popularity',
    'apiKey':newsAPI_key
  }       

  if (q):
    search_params['q'] = q
  if (source_id):
    search_params['sources'] = source_id
  if (from_date):
    search_params['from'] = from_date
  if (to_date):
    search_params['to'] = to_date
  if (limit):
    search_params['pageSize'] = limit
    
  response = requests.get(url, search_params)
  df = pd.DataFrame()
  
  if (response.json()['status']=='error'):
    print(response.json()['code'],response.json()['message'])
    return(df)
  else:    
    articles_json = response.json()['articles']
    if (bool(response.json()['articles'])):
      for x in response.json()['articles']:
        article_dict = {}
        try:
          article_dict['source_name'] = x['source']['name']
        except:
          pass
        try:
          article_dict['source_id'] = x['source']['id']
        except:
          pass
        try:
          date = datetime.strptime(x['publishedAt'], '%Y-%m-%dT%H:%M:%SZ')
          article_dict['date'] = str(date.year) + '-' + str(date.month) + '-' + str(date.day)
          article_dict['year'] = date.year
          article_dict['month'] = date.month
          article_dict['day'] = date.day
        except:
          pass
        try:
          article_dict['title'] = x['title']
        except:
          pass
        try:
          article_dict['domain'] = urlparse(x['url']).netloc
        except:
          pass
        try:
          article_dict['urlToImage'] = x['urlToImage']
        except:
          continue
          
        df = df.append(article_dict, ignore_index=True)

  return(df)


In [12]:
temp_dict = {"olddate":"2020-10-21T06:12:16Z"}


date = datetime.strptime(temp_dict['olddate'], '%Y-%m-%dT%H:%M:%SZ')

temp_dict['date'] = str(date.year) + '-' + str(date.month) + '-' + str(date.day)
temp_dict['year'] = date.year
temp_dict['month'] = date.month
temp_dict['day'] = date.day
temp_dict['hour'] = date.hour
temp_dict['minute'] = date.minute

temp_dict


Out[119]: {'olddate': '2020-10-21T06:12:16Z',
 'date': '2020-10-21',
 'year': 2020,
 'month': 10,
 'day': 21,
 'hour': 6,
 'minute': 12}

In [13]:

newsAPI_key = '802c3d598bfe418f9165b8a1d655479e'

oldest = str(datetime.today().year) + '-' + str(datetime.today().month) + '-' + str(datetime.today().day-7)
newest = str(datetime.today().year) + '-' + str(datetime.today().month) + '-' + str(datetime.today().day)

temp_df = getArticleData(source_id='cnn', q='biden', from_date=oldest, to_date=newest, limit=1)


In [14]:

# newsAPI_key = '467ee87cae8346669f5cf44c9e9a5c57'
# newsAPI_key = 'a28ebf83077c40158d5b9d19cbaa97f3'
newsAPI_key = '05bbf397f9b04b8185dcb00b77f1c429'
# newsAPI_key = '802c3d598bfe418f9165b8a1d655479e'
# newsAPI_key = 'fbd13bdbb4d74fddbd53365207fdadad'
# newsAPI_key = 'ab5008e3358b4e188019c24a468df86a'
# newsAPI_key = '294e9cf5c8ba4f328c9dc35b75c4383c'


q_list = ['trump','biden']

date_range = 7   # how many recent days to include in search results

oldest = str(datetime.today().year) + '-' + str(datetime.today().month) + '-' + str(datetime.today().day - date_range)
newest = str(datetime.today().year) + '-' + str(datetime.today().month) + '-' + str(datetime.today().day)

article_df = pd.DataFrame()

for query in q_list:
  for s in source_id_list:
    article_df = article_df.append(getArticleData(source_id=s, 
                                                  q=query, 
                                                  from_date=oldest, 
                                                  to_date=newest, 
                                                  limit=50))

display(article_df)


date,day,domain,month,source_id,source_name,title,urlToImage,year
2020-10-23,23.0,abcnews.go.com,10.0,abc-news,ABC News,WATCH: Biden and Trump speak on institutional racism in America,https://s.abcnews.com/images/Politics/201022_abcnl_debate_race_issues_hpMain_16x9_992.jpg,2020.0
2020-10-20,20.0,abcnews.go.com,10.0,abc-news,ABC News,"Trump targets Pennsylvania women, but will do so without wife Melania",https://s.abcnews.com/images/Politics/trump-20-rt-er-190618_hpMain_16x9_992.jpg,2020.0
2020-10-22,22.0,abcnews.go.com,10.0,abc-news,ABC News,How will Biden deal with Trump's personal attacks at Thursday's debate?,https://s.abcnews.com/images/Politics/biden-8-abc-er-201015_1602814151696_hpMain_16x9_992.jpg,2020.0
2020-10-23,23.0,abcnews.go.com,10.0,abc-news,ABC News,5 key takeaways from the final presidential debate,https://s.abcnews.com/images/Politics/trump-biden-debate-nashville-37-rtr-jc-201022_1603419468680_hpMain_16x9_992.jpg,2020.0
2020-10-20,20.0,abcnews.go.com,10.0,abc-news,ABC News,"US, Russia move closer to deal to extend last nuclear arms control pact",https://s.abcnews.com/images/Politics/donald-trump-vladimir-putin-ap-mt-180721_hpMain_16x9_992.jpg,2020.0
2020-10-26,26.0,abcnews.go.com,10.0,abc-news,ABC News,"Jews For Trump car parade stirs protests, fights in New York",null,2020.0
2020-10-23,23.0,abcnews.go.com,10.0,abc-news,ABC News,Biden campaign seeks to clarify position on fossil fuels after debate exchange,https://s.abcnews.com/images/Politics/trump-biden-debate-nashville-12-gty-llr-201022_1603420778438_hpMain_16x9_992.jpg,2020.0
2020-10-24,24.0,abcnews.go.com,10.0,abc-news,ABC News,"Lawyer for Ivanka Trump, Jared Kushner threatens suit over 'defamatory' billboards",https://s.abcnews.com/images/Politics/ivanka-trump-jared-kushner-times-square-billboard-01-gty-llr-201024_1603557674132_hpMain_16x9_992.jpg,2020.0
2020-10-23,23.0,abcnews.go.com,10.0,abc-news,ABC News,"For Trump and Biden, debate fuels allegations of improper family foreign deals",https://s.abcnews.com/images/Politics/trump-biden-debate-nashville-24-rtr-jc-201022_1603416146080_hpMain_16x9_992.jpg,2020.0
2020-10-23,23.0,abcnews.go.com,10.0,abc-news,ABC News,Trump lifts sanctions on Sudan as he announces deal between African nation and Israel,https://s.abcnews.com/images/Politics/trump-gty-jt-201016_1602863137431_hpMain_16x9_992.jpg,2020.0


In [15]:
article_df = article_df.drop_duplicates('title') # drop duplicate articles
article_df = article_df.loc[article_df.urlToImage != 'null',:] # drop null URL strings
article_df = article_df.reset_index(drop=True)


In [16]:
article_df.to_csv('/dbfs/saved_df/article_df.csv')

In [17]:
test_df = pd.read_csv("/dbfs/saved_df/article_df.csv", header=0, index_col=0)
display(test_df)

date,day,domain,month,source_id,source_name,title,urlToImage,year
2020-10-23,23.0,abcnews.go.com,10.0,abc-news,ABC News,WATCH: Biden and Trump speak on institutional racism in America,https://s.abcnews.com/images/Politics/201022_abcnl_debate_race_issues_hpMain_16x9_992.jpg,2020.0
2020-10-20,20.0,abcnews.go.com,10.0,abc-news,ABC News,"Trump targets Pennsylvania women, but will do so without wife Melania",https://s.abcnews.com/images/Politics/trump-20-rt-er-190618_hpMain_16x9_992.jpg,2020.0
2020-10-22,22.0,abcnews.go.com,10.0,abc-news,ABC News,How will Biden deal with Trump's personal attacks at Thursday's debate?,https://s.abcnews.com/images/Politics/biden-8-abc-er-201015_1602814151696_hpMain_16x9_992.jpg,2020.0
2020-10-23,23.0,abcnews.go.com,10.0,abc-news,ABC News,5 key takeaways from the final presidential debate,https://s.abcnews.com/images/Politics/trump-biden-debate-nashville-37-rtr-jc-201022_1603419468680_hpMain_16x9_992.jpg,2020.0
2020-10-20,20.0,abcnews.go.com,10.0,abc-news,ABC News,"US, Russia move closer to deal to extend last nuclear arms control pact",https://s.abcnews.com/images/Politics/donald-trump-vladimir-putin-ap-mt-180721_hpMain_16x9_992.jpg,2020.0
2020-10-23,23.0,abcnews.go.com,10.0,abc-news,ABC News,Biden campaign seeks to clarify position on fossil fuels after debate exchange,https://s.abcnews.com/images/Politics/trump-biden-debate-nashville-12-gty-llr-201022_1603420778438_hpMain_16x9_992.jpg,2020.0
2020-10-24,24.0,abcnews.go.com,10.0,abc-news,ABC News,"Lawyer for Ivanka Trump, Jared Kushner threatens suit over 'defamatory' billboards",https://s.abcnews.com/images/Politics/ivanka-trump-jared-kushner-times-square-billboard-01-gty-llr-201024_1603557674132_hpMain_16x9_992.jpg,2020.0
2020-10-23,23.0,abcnews.go.com,10.0,abc-news,ABC News,"For Trump and Biden, debate fuels allegations of improper family foreign deals",https://s.abcnews.com/images/Politics/trump-biden-debate-nashville-24-rtr-jc-201022_1603416146080_hpMain_16x9_992.jpg,2020.0
2020-10-23,23.0,abcnews.go.com,10.0,abc-news,ABC News,Trump lifts sanctions on Sudan as he announces deal between African nation and Israel,https://s.abcnews.com/images/Politics/trump-gty-jt-201016_1602863137431_hpMain_16x9_992.jpg,2020.0
2020-10-26,26.0,abcnews.go.com,10.0,abc-news,ABC News,Kushner says Black people must 'want to be successful',https://s.abcnews.com/images/Politics/WireAP_56e23eecae8f47cea9bcc524b04e8e2e_16x9_992.jpg,2020.0


In [18]:
%sh git clone https://github.com/adichiara/DS504

fatal: destination path 'DS504' already exists and is not an empty directory.

total 552
drwxr-xr-x 3 root root 4096 Oct 27 16:52 .
drwxr-xr-x 1 root root 4096 Oct 27 16:43 ..
-rwxr-xr-x 1 root root 272851 Oct 27 16:53 article_df.csv
drwxr-xr-x 8 root root 4096 Oct 27 16:43 .git
-rw-r--r-- 1 root root 7 Oct 27 16:43 README.md
-rw-r--r-- 1 root root 272851 Oct 27 16:51 source_data.csv